# QMMM workflow using GROMACS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use methane as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://votca.github.io/xtp_map.html)
* [xtp_parallel](https://votca.github.io/xtp_parallel.html)
* [xtp_run](https://votca.github.io/xtp_run.html)
* [xtp_tools](https://votca.github.io/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [ ]:
!xtp_tools -h

### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Create a folder to store the input `Options` for XTP

In [ ]:
!mkdir -p OPTIONFILES

Remove previous hdf5 file

In [ ]:
!rm -f state.hdf5

Import an small python module to edit xml files

In [ ]:
from xml_editor import add_section, edit_calculator
import os

## Generate the topology from the Gromacs file

runs the mapping from MD coordinates to segments and creates an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/). You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/).

In [ ]:
!xtp_map -t MD_FILES/topol.tpr -c MD_FILES/conf.gro -s system.xml -f state.hdf5

##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. We will need to copy the input from the `VOTCA` installation path to our local copy.

In [ ]:
!cp "$VOTCASHARE/xtp/xml/mapchecker.xml" OPTIONFILES/

The `${VOTCASHARE}` environmental variable is set to the path that you provided during the `VOTCA` [installation](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md), by the default is set to `/usr/local/votca`.

We need to update the `mapchecker` options with the file containing the definition of the system. For doing, so we will use the python module `xml_editor` that we have imported above. The first argument is the name of the calculator (i.e. `mapchecker`), the seconds argument is the name of the option to change and the third one, the value to use.

In [ ]:
edit_calculator("mapchecker", "map_file", "system.xml")

In the [mapchecker section of the manual](https://votca.github.io/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [ ]:
!xtp_run -e mapchecker -o OPTIONFILES/mapchecker.xml -f state.hdf5

## Neighborlist Calculation

The following step is to determine the neighbouring pairs for exciton transport. We will need first to get a copy of the input 

In [ ]:
cp "$VOTCASHARE/xtp/xml/neighborlist.xml" OPTIONFILES/

Then we will set the *cutoff* for the pairs to 0.5 nanometers. See the [neighborlist options](https://votca.github.io/neighborlist.html) for further information. 

In [ ]:
exciton_cutoff = """
<exciton_cutoff>0.5</exciton_cutoff>
"""
segments = """
<segments>
  <type>DCV5T_ME3 DCV5T_ME3</type>
  <cutoff>3</cutoff>
</segments>
"""
constant = """<constant help="cutoff for all other types of pairs">0.6</constant>"""
add_section("neighborlist", exciton_cutoff)
add_section("neighborlist", segments)
add_section("neighborlist", constant)

Finally, we can run the calculation using 4 threads

In [ ]:
!xtp_run -e neighborlist -o OPTIONFILES/neighborlist.xml -f state.hdf5

## Read reorganization energies
In this step we will read the in site reorganization energies and store them in the `state.hdf5` file. We just need to copy the input file and execute the calculation

In [ ]:
!cp "$VOTCASHARE/xtp/xml/einternal.xml" OPTIONFILES/
!xtp_run -e einternal -o OPTIONFILES/einternal.xml -f state.hdf5

## Compute site energy
In this step we will perform some *QMMM* calculations to compute the site energies. The `qmmm_mm.xml` file contains some predefined settings to perform the *MM* calculations. Let us first copy these settings into the state file,

In [ ]:
!cp qmmm_mm.xml OPTIONFILES
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "write"

The previous command generates a `qmmm_mm_jobs.xml` containing 4000 *MM* jobs to compute, if you examine that file, it should look something like

```xml
<jobs>
  <job>
    <id>0</id>
    <tag>Methane_0:n</tag>
    <input>
      <site_energies>0:n</site_energies>
      <regions>
        <region>
          <id>0</id>
          <segments>0:n</segments>
        </region>
      </regions>
    </input>
    <status>AVAILABLE</status>
  </job>    
...
```

Let us run just the first 4 jobs by settings all jobs `status` to `COMPLETE` except for the first four. This can be easily done with [sed](https://www.gnu.org/software/sed/manual/sed.html) as follows,

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml

Now we can run the jobs and save the results in the state file

In [ ]:
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -p 2 -j "run"
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "read"

## Site energy and pair energy analysis
In this step we generate an histogram and compute the correlation function of site energies and pair energy differences.

In [ ]:
!cp "$VOTCASHARE/xtp/xml/eanalyze.xml" OPTIONFILES/
!xtp_run -e eanalyze -o OPTIONFILES/eanalyze.xml -f state.hdf5

You should now see a set of files prefixed with `eanalyze` containing the histrogram and correlation functions.

In [ ]:
!ls eanalyze*

## QM energy calculation
Our next task is to perform the qm calculations for each segment that we have stored in the hdf5 file. The calculations take place in 3 stages: write the jobs to a file, perform the computation and finally save the results to the state file. We will first copy the input into our local folder

In [ ]:
!cp "$VOTCASHARE/xtp/xml/eqm.xml" OPTIONFILES/

Now we set the GWBSE mode to `G0W0`,  the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. For more information, check the [eqm calculator options](https://votca.github.io/eqm.html).

In [ ]:
edit_calculator("eqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("eqm", "gwbse_options.gwbse.mode", "G0W0")
edit_calculator("eqm", "gwbse_options.gwbse.qp_solver", "fixedpoint")
edit_calculator("eqm", "gwbse_options.gwbse.qp_grid_steps", "100")
edit_calculator("eqm", "gwbse_options.gwbse.qp_grid_spacing", "0.01")
edit_calculator("eqm", "gwbse_options.gwbse.exctotal", "10")
edit_calculator("eqm", "map_file", "system.xml")
edit_calculator("eqm", "basisset", "3-21G")
edit_calculator("eqm", "auxbasisset", "aux-def2-svp")

For the sake of computational time let just compute the `gw` approximation and the `singlet`. You can also request the `triplet` or `all`,  see the [gwbse sectionfor the eqm calculator](https://votca.github.io/eqm.html).

In [ ]:
edit_calculator("eqm", "gwbse_options.gwbse.tasks", "gw,singlets")

First we will write the job in a file and enable only the first 2,

In [ ]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -s 0 -j "write"
!sed -i "s/AVAILABLE/COMPLETE/g" eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs


Now, let run these 2 jobs

In [ ]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -p 2 -s 0 -j run -c 1

## QM calculation for pairs
In the following step we will run QM calculations for each pair in the hdf5 file. As the calculations on the previous step, we will first write the jobs in a file, then run them and finally store the results in the state file. First, we need to copy the input to our local folder

In [ ]:
!cp "$VOTCASHARE/xtp/xml/iqm.xml" OPTIONFILES/

As in the previous section, we set the GWBSE mode to `G0W0`, the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. But we compute only the `gw` approximation. We are also going to compute a single excited state. For more information, check the [iqm calculator options](https://votca.github.io/iqm.html). We also want to compute the `singlet` couplings. 

In [ ]:
edit_calculator("iqm", "states", "1")
edit_calculator("iqm", "map_file", "system.xml")
edit_calculator("iqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("iqm", "gwbse_options.gwbse.tasks", "gw")
edit_calculator("iqm", "gwbse_options.gwbse.qp_solver", "fixedpoint")
edit_calculator("iqm", "gwbse_options.gwbse.qp_grid_steps", "100")
edit_calculator("iqm", "gwbse_options.gwbse.qp_grid_spacing", "0.01")
edit_calculator("iqm", "gwbse_options.gwbse.exctotal", "10")
edit_calculator("iqm", "bsecoupling.spin", "singlet")
edit_calculator("iqm", "basisset", "3-21G")
edit_calculator("iqm", "auxbasisset", "aux-def2-svp")

Before running the calculations, we need to specify in the `iqm` input which states to read into the jobfile for each segment type. The following *XML* snippet needs to be added

In [ ]:
readjobfile = """
<readjobfile help="which states to read into the jobfile for each segment type">
     <singlet>Methane:s1</singlet>
     <triplet>Methane:t1</triplet>
     <electron>Methane:e1</electron>
     <hole>Methane:h1</hole>
</readjobfile>
"""

The following code add the `readjobfile` section,

In [ ]:
add_section("iqm", readjobfile)

Now, let's write the jobs to the file

In [ ]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -s 0 -j "write"

From the jobs that we just write down, let's make available only the first job

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" iqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' iqm.jobs

Now we can run and store the jobs results

In [ ]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -p 2 -s 0 -j run -c 1

Finally, we read the results into the state

In [ ]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -j "read"

## Coupling
We can now compute the classical coupling of transition in the aformentioned three stages,

In [ ]:
!cp "${VOTCASHARE}/xtp/xml/iexcitoncl.xml" OPTIONFILES

We need to change in the `iexcitoncl` input the name `map_file` option and add the state. check all the available of the [iexcitoncl calculator](https://votca.github.io/iexcitoncl.html).

In [ ]:
edit_calculator("iexcitoncl", "map_file", "system.xml")

In [ ]:
state = "<states>Methane:n2s1</states>"
add_section("iexcitoncl", state)

In [ ]:
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "write"

Now we can run and save the jobs. For demo purposes we will run only the first job

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" exciton.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' exciton.job
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5

In [ ]:
#!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "read"

## Coupling analysis
Using the coupling computed in the previous steps, we will generate an histogram for the squared couplings in logarithmic scale,

In [ ]:
!cp "${VOTCASHARE}/xtp/xml/ianalyze.xml" OPTIONFILES
edit_calculator("ianalyze", "states", "e,h,s")
!xtp_run -e ianalyze -o OPTIONFILES/ianalyze.xml -f state.hdf5

## QMMM calculations

In [ ]:
!cp qmmm.xml OPTIONFILES/
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "write"

Lets run just the first job

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_jobs.xml                                                                                                                                                            
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml                                                                                                                                                 
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -p 2 -f state.hdf5 -j run

Finally, save the results

In [ ]:
#!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "read"